## **ECONOMIC ANALYSIS**

In [1]:
%run ./_setup.py
import os
import sys

Python: c:\Users\Admin\Desktop\Irrigation Water Requirement Prediction\.venv\Scripts\python.exe
Environment ready
Shape: (10000, 20)
<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Soil_Type                10000 non-null  str    
 1   Soil_pH                  10000 non-null  float64
 2   Soil_Moisture            10000 non-null  float64
 3   Organic_Carbon           10000 non-null  float64
 4   Electrical_Conductivity  10000 non-null  float64
 5   Temperature_C            10000 non-null  float64
 6   Humidity                 10000 non-null  float64
 7   Rainfall_mm              10000 non-null  float64
 8   Sunlight_Hours           10000 non-null  float64
 9   Wind_Speed_kmh           10000 non-null  float64
 10  Crop_Type                10000 non-null  str    
 11  Crop_Growth_Stage        10000 non-null  str    
 12  Season   

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
df = pd.read_csv("../data/irrigation_prediction.csv")

In [3]:
# Ensure categorical columns are properly typed
categorical_cols = ['Soil_Type', 'Crop_Type', 'Crop_Growth_Stage', 'Season', 
                    'Irrigation_Type', 'Water_Source', 'Mulching_Used', 'Region', 
                    'Irrigation_Need']

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')
    else:
        print(f"Warning: Column '{col}' not found in dataset")

In [4]:
# Crop prices (example market prices)
crop_prices = {
    'Wheat': 250, 'Maize': 220, 'Cotton': 500, 
    'Rice': 300, 'Sugarcane': 280, 'Potato': 200
}
df['Crop_Price'] = df['Crop_Type'].map(crop_prices)

### *76. Profitability vs Water Use*

In [6]:
print("\n76. Water Profitability...")

df["Crop_Price"] = pd.to_numeric(df["Crop_Price"], errors="coerce")
df["Previous_Irrigation_mm"] = pd.to_numeric(df["Previous_Irrigation_mm"], errors="coerce")

df["Revenue_per_ha"] = df["Crop_Price"] * 10  # Simplified yield
df["Profit_per_ha"] = df["Revenue_per_ha"] - df["Previous_Irrigation_mm"] * 0.1  # Cost of water
df["Water_Productivity"] = df["Revenue_per_ha"] / (df["Previous_Irrigation_mm"] + 1)


fig76 = px.scatter(df, x='Previous_Irrigation_mm', y='Profit_per_ha',
                   color='Crop_Type', size='Field_Area_hectare',
                   title='<b>76. Profitability vs Water Use</b><br>Size = Field Area',
                   labels={'Previous_Irrigation_mm': 'Water Used (mm)', 
                          'Profit_per_ha': 'Profit per ha ($)'})

fig76.update_layout(height=500, plot_bgcolor='white')
fig76.show()


76. Water Profitability...


### *77. Cost-Benefit Analysis*

In [7]:
print("\n77. Cost-Benefit Analysis...")

cost_benefit = df.groupby('Crop_Type', observed=False).agg({
    'Revenue_per_ha': 'mean',
    'Previous_Irrigation_mm': 'mean',
    'Water_Productivity': 'mean'
}).reset_index()

cost_benefit['Water_Cost'] = cost_benefit['Previous_Irrigation_mm'] * 0.1
cost_benefit['Net_Profit'] = cost_benefit['Revenue_per_ha'] - cost_benefit['Water_Cost']

fig77 = go.Figure()
fig77.add_trace(go.Bar(name='Revenue', x=cost_benefit['Crop_Type'], 
                       y=cost_benefit['Revenue_per_ha']))
fig77.add_trace(go.Bar(name='Water Cost', x=cost_benefit['Crop_Type'], 
                       y=cost_benefit['Water_Cost']))

fig77.update_layout(barmode='group', title='<b>77. Revenue vs Water Cost by Crop</b>',
                   height=500, plot_bgcolor='white')
fig77.show()


77. Cost-Benefit Analysis...


### *78. Price Volatility Impact*

In [8]:
print("\n78. Price Volatility...")

# Simulate price variations
price_scenarios = []
for crop in crop_prices.keys():
    for variation in [-20, -10, 0, 10, 20]:  # Price changes in %
        price_scenarios.append({
            'Crop': crop,
            'Price_Change': variation,
            'Revenue_Impact': crop_prices[crop] * (1 + variation/100) * 10
        })

price_df = pd.DataFrame(price_scenarios)

fig78 = px.line(price_df, x='Price_Change', y='Revenue_Impact',
                color='Crop', markers=True,
                title='<b>78. Price Sensitivity by Crop</b>',
                labels={'Price_Change': 'Price Change (%)', 
                       'Revenue_Impact': 'Revenue per ha ($)'})

fig78.update_layout(height=500, plot_bgcolor='white')
fig78.show()


78. Price Volatility...


### *79. Subsidy Effectiveness*

In [11]:
print("\n79. Subsidy Impact...")

# Simulate subsidy scenarios
df['With_Subsidy'] = df['Previous_Irrigation_mm'] * 0.05  # 50% water cost subsidy
df['Profit_With_Subsidy'] = df['Revenue_per_ha'] - (df['Previous_Irrigation_mm'] * 0.05)

subsidy_impact = df.groupby('Field_Area_hectare', observed=False).agg({
    'Profit_per_ha': 'mean',
    'Profit_With_Subsidy': 'mean'
}).reset_index()

fig79 = px.line(subsidy_impact, x='Field_Area_hectare', y=['Profit_per_ha', 'Profit_With_Subsidy'],
                title='<b>79. Subsidy Impact by Farm Size</b>',
                labels={'value': 'Profit per ha ($)', 
                       'Field_Area_hectare': 'Field Area (ha)',
                       'variable': 'Scenario'})

fig79.show()


79. Subsidy Impact...


### *80. Market Price Correlation*

In [13]:
print("\n80. Market Correlations...")

price_correlations = df.groupby('Season', observed=False).agg({
    'Crop_Price': 'mean',
    'Previous_Irrigation_mm': 'mean',
    'Water_Productivity': 'mean'
}).reset_index()

fig80 = px.scatter_matrix(price_correlations, 
                         dimensions=['Crop_Price', 'Previous_Irrigation_mm', 'Water_Productivity'],
                         color='Season',
                         title='<b>80. Market & Water Use Correlations by Season</b>')

fig80.update_layout(height=600, plot_bgcolor='black')
fig80.show()


80. Market Correlations...


### *81. Export Potential Analysis*

In [14]:
print("\n81. Export Potential...")

# Simplified export potential score
export_potential = df.groupby('Crop_Type', observed=False).agg({
    'Crop_Price': 'mean',
    'Water_Productivity': 'mean',
    'Field_Area_hectare': 'sum'
}).reset_index()

export_potential['Export_Score'] = (
    export_potential['Crop_Price'] * 0.4 +
    export_potential['Water_Productivity'] * 0.3 +
    (export_potential['Field_Area_hectare'] / export_potential['Field_Area_hectare'].max()) * 0.3
) * 100

fig81 = px.treemap(export_potential, path=['Crop_Type'], 
                   values='Field_Area_hectare', color='Export_Score',
                   color_continuous_scale='RdYlGn',
                   title='<b>81. Export Potential by Crop</b><br>Size = Total Area, Color = Export Score')

fig81.update_layout(height=500)
fig81.show()


81. Export Potential...


### *82. Value Addition Opportunities*

In [15]:
print("\n82. Value Addition...")

value_chains = {
    'Wheat': ['Flour', 'Bread', 'Pasta'],
    'Maize': ['Cornflakes', 'Starch', 'Ethanol'],
    'Cotton': ['Textiles', 'Clothing', 'Oil'],
    'Rice': ['Basmati', 'Flour', 'Snacks'],
    'Sugarcane': ['Sugar', 'Molasses', 'Ethanol'],
    'Potato': ['Chips', 'Starch', 'Frozen']
}

value_data = []
for crop, products in value_chains.items():
    for i, product in enumerate(products):
        value_data.append({
            'Crop': crop,
            'Product': product,
            'Value_Multiplier': 1 + (i+1) * 0.5  # Increasing value addition
        })

value_df = pd.DataFrame(value_data)

fig82 = px.bar(value_df, x='Crop', y='Value_Multiplier', color='Product',
               barmode='group', title='<b>82. Value Addition Potential</b>',
               labels={'Value_Multiplier': 'Value Multiplier'})

fig82.update_layout(height=500, plot_bgcolor='white')
fig82.show()


82. Value Addition...


### *83. Employment Generation*

In [17]:
print("\n83. Employment Analysis...")

# Labor coefficients (person-days per hectare)
labor_coeff = {'Wheat': 50, 'Maize': 60, 'Cotton': 80, 
               'Rice': 70, 'Sugarcane': 90, 'Potato': 65}

employment = df.groupby(["Crop_Type", "Region"], observed=False).agg(
    Field_Area_hectare=("Field_Area_hectare", "sum"),
).reset_index()

employment["Labor_Days"] = (
    employment["Crop_Type"].map(labor_coeff).astype(float)
    * pd.to_numeric(employment["Field_Area_hectare"], errors="coerce")
)

fig83 = px.bar(employment, x='Region', y='Labor_Days', color='Crop_Type',
               title='<b>83. Employment Generation by Region & Crop</b>',
               labels={'Labor_Days': 'Total Labor Days'})

fig83.update_layout(height=500, plot_bgcolor='white')
fig83.show()


83. Employment Analysis...


### *84. Investment Priority Matrix*

In [21]:
print("\n84. Investment Priorities...")

investment_data = []
for _, row in cost_benefit.iterrows():
    investment_data.append({
        'Crop': row['Crop_Type'],
        'Impact': row['Water_Productivity'] * 100,  # Water efficiency impact
        'Cost': row['Previous_Irrigation_mm'] * 0.1  # Water cost
    })

invest_df = pd.DataFrame(investment_data)

fig84 = px.scatter(invest_df, x='Cost', y='Impact', color='Crop',
                   size='Impact', title='<b>84. Investment Priority Matrix</b><br>Quick Wins (High Impact, Low Cost)',
                   labels={'Cost': 'Water Cost ($/ha)', 'Impact': 'Water Efficiency Impact'})

# Add quadrants
fig84.add_hline(y=invest_df['Impact'].median(), line_dash="dash", line_color="red")
fig84.add_vline(x=invest_df['Cost'].median(), line_dash="dash", line_color="red")

fig84.update_layout(height=500, plot_bgcolor='black')
fig84.show()


84. Investment Priorities...


### *85. Economic Resilience Index*

In [22]:
print("\n85. Economic Resilience...")

resilience = df.groupby('Region', observed=False).agg({
    'Crop_Price': 'mean',
    'Water_Productivity': 'mean',
    'Crop_Type': 'nunique',  # Crop diversity
    'Profit_per_ha': 'std'  # Profit stability
}).reset_index()

# Calculate resilience score (higher is better)
resilience['Price_Score'] = resilience['Crop_Price'] / resilience['Crop_Price'].max() * 100
resilience['Efficiency_Score'] = resilience['Water_Productivity'] / resilience['Water_Productivity'].max() * 100
resilience['Diversity_Score'] = resilience['Crop_Type'] / resilience['Crop_Type'].max() * 100
resilience['Stability_Score'] = 100 - (resilience['Profit_per_ha'] / resilience['Profit_per_ha'].max() * 100)

resilience['Resilience_Score'] = (
    resilience['Price_Score'] * 0.25 +
    resilience['Efficiency_Score'] * 0.25 +
    resilience['Diversity_Score'] * 0.25 +
    resilience['Stability_Score'] * 0.25
)

fig85 = px.bar(resilience.sort_values('Resilience_Score'), 
               x='Resilience_Score', y='Region', orientation='h',
               color='Resilience_Score', color_continuous_scale='RdYlGn',
               title='<b>85. Economic Resilience by Region</b>',
               labels={'Resilience_Score': 'Economic Resilience Score'})

fig85.update_layout(height=500, plot_bgcolor='white')
fig85.show()


85. Economic Resilience...
